In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import polars as pl
import os
import glob
import shutil
import zipfile
import librosa
from IPython.display import Audio
import pickle
from joblib import dump, load
from pathlib import Path
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score

In [2]:
config = {}
Test = True

if Test:
    config['root_file_path'] = '/kaggle/input/birdclef-2024/unlabeled_soundscapes/'
    config['start'] = 7000
    config['end'] = 8200
    filenames_with_path = glob.glob(f"{config['root_file_path']}*.ogg")
    filenames = [os.path.basename(filename) for filename in filenames_with_path][config['start']:config['end']]
    print(len(filenames))
else:
    config['root_file_path'] = '/kaggle/input/birdclef-2024/test_soundscapes/'
    filenames_with_path = glob.glob(f"{config['root_file_path']}*.ogg")
    filenames = [os.path.basename(filename) for filename in filenames_with_path]
    print(len(filenames))

1200


In [ ]:
from tqdm import tqdm

chunk_feature_dict = {}

for file in tqdm(filenames):
    full_path = config['root_file_path']+file
    audio, sample_rate = librosa.load(path=full_path, sr=32000)
    cur_file = file.replace('.ogg','')
    samples_per_segment = sample_rate * 5
    if len(audio) > 7680000:
        total_samples = 7680000
    else:
        total_samples = len(audio)
        
    for i in range(0, total_samples+160000, samples_per_segment):
        if i + samples_per_segment <= total_samples:
            segment = audio[i:i + samples_per_segment]
            chunk = cur_file+'_'+str(int((i/32000)+5))
            mfccs = librosa.feature.mfcc(y=segment, sr=32000, n_mfcc=40)
            flattened_features = (np.mean(mfccs.T, axis=0)).reshape(1, -1)
            chunk_feature_dict[chunk] = np.ascontiguousarray(flattened_features)

  7%|▋         | 81/1200 [02:29<29:55,  1.60s/it] 

In [ ]:
meta_data = pl.read_csv('../input/birdclef-2024/train_metadata.csv', low_memory=True)
bird_cols = list(meta_data['primary_label'].unique().sort())

submit = pd.read_csv("/kaggle/input/birdclef-2024/sample_submission.csv")
submit['row_id'] = 'samples'

submit

In [ ]:
rbf_kernel_svm_model = load('/kaggle/input/rbf-kernel-svc-model/rbf_kernel_svm_model.joblib')

In [ ]:
for file_name, feature_set in tqdm(chunk_feature_dict.items()):
    df = pd.DataFrame(rbf_kernel_svm_model.predict_proba(feature_set), columns=bird_cols)
    df.insert(loc=0, column='row_id', value=file_name)
    submit = pd.concat([submit,df]).reset_index(drop=True)
    
i = submit[(submit.row_id == 'samples')].index
submit = submit.drop(i).reset_index(drop=True)

In [ ]:
sample_submission = pd.read_csv("/kaggle/input/birdclef-2024/sample_submission.csv")
assert set(sample_submission.columns) == set(submit.columns)

In [ ]:
submit.to_csv('submission.csv', index=False)